In [ ]:
# !pip install requests
# !pip install Pillow
# !pip install deepface
# !pip install pandas

In [ ]:
import os
import numpy as np
from deepface import DeepFace
import pandas as pd
import requests
from PIL import Image
from io import BytesIO

# Read CSV file
df = pd.read_csv("input.csv")

# Separate dataframes for male and female
df_male = df[df["Gender"] == "Male"]
df_female = df[df["Gender"] == "Female"]

dir_male = "images/male/"
dir_female = "images/female/"

# Create directories if they don't exist
os.makedirs(dir_male, exist_ok=True)
os.makedirs(dir_female, exist_ok=True)

# Counters for male and female names
male_counter = 0
female_counter = 0

while male_counter < len(df_male) or female_counter < len(df_female):
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36"
    }
    response = requests.get(
        "https://www.thispersondoesnotexist.com", headers=headers)

    if response.status_code == 200 and "image" in response.headers["Content-Type"]:
        img = Image.open(BytesIO(response.content))

        # Crop the image
        width, height = img.size
        img = img.crop((20, 20, width - 20, height - 20))

        # Convert the PIL Image to a numpy array
        img_np = np.array(img)

        # Predict gender using DeepFace
        predictions = DeepFace.analyze(img_np, actions=['gender'])

        # Determine the gender based on the higher probability
        predicted_gender = "Man" if predictions[0]['gender']['Man'] > predictions[0]['gender']['Woman'] else "Woman"

        # Save image in the appropriate directory
        if predicted_gender == "Man" and male_counter < len(df_male):
            file_name = df_male.iloc[male_counter]["Name"]
            full_file_name = dir_male + file_name + ".jpg"
            male_counter += 1
            img.save(full_file_name)
            print(
                f"Image {file_name} is predicted to be {predicted_gender} and saved to {full_file_name}"
            )
        elif predicted_gender == "Woman" and female_counter < len(df_female):
            file_name = df_female.iloc[female_counter]["Name"]
            full_file_name = dir_female + file_name + ".jpg"
            female_counter += 1
            img.save(full_file_name)
            print(
                f"Image {file_name} is predicted to be {predicted_gender} and saved to {full_file_name}"
            )
    else:
        print(
            f"Failed to download image. Status code: {response.status_code}, Content-Type: {response.headers['Content-Type']}"
        )